ECE 8803 Final Project

In [97]:
import numpy as np
import pandas as pd
import os

'''Obtain Dataset'''
root = os.getcwd()
train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")
trainset = pd.read_csv(train).dropna()
testset = pd.read_csv(test).dropna()

LABELS_SEVERITY = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}

'''Train Data'''
X_train = trainset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
Y_train = trainset[["DRSS"]].to_numpy()
y = []
for i in range(Y_train.shape[0]):
    y.append(LABELS_SEVERITY[int(Y_train[i])])

Y_train = np.array(y)



'''Test Data'''
X_test = testset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
Y_test = testset[["DRSS"]].to_numpy() 
y = []
for i in range(Y_test.shape[0]):
    y.append(LABELS_SEVERITY[int(Y_test[i])])

Y_test = np.array(y)




Naive Bayes Implementation

In [98]:
# implement naive bayes class for PRIME_FULL

import numpy as np
#import sklearn
#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class MyNaiveBayes:
    def __init__(self, X_train, y_train):
        """Function intializes the Naive Bayes class.
        
        Parameters:
        -----------
        X_train: ndarray of shape (N,D)
            Numpy array containing N training examples, each D dimensional.
        
        y_train: ndarray of shape (N,)
            Numpy array containing vector of ground truth classes for examples 
            in X_train
        """
        
        self.X_train = X_train
        self.y_train = y_train
        
    ## quetion i
    def log_gaussian(self, x, mean, std):
        """Function computes log P(x) from a normal distribution specified by 
        parameters mean and std. To be called during inference"""
        
        log_likelihood = -np.log(std * np.sqrt(2 * np.pi)) - (0.5 * ((x-mean) / std) ** 2)    # implement code to compute log gaussian value for 
                                   # conditional probabilities 
        
        return log_likelihood
    
    ## quetion ii
    def fit(self):
        """Function computes likelihood parameters from training data in the \
        training phase"""

        # calculate class priors from train data
        data_length = self.X_train.shape[0]
        num_features = self.X_train.shape[1]
        
        C, Nc = np.unique(self.y_train, return_counts=True)
        num_classes = len(C)
        self.priors = Nc / data_length 
        #print(self.priors)
        
        # calculate per class data likelihoods
        
        self.feature_means = np.zeros((num_classes, num_features))
        self.feature_std = np.zeros((num_classes, num_features))
        
        #print(type(self.y_train[0]))
        for i in range(len(C)):
          self.feature_means[i] = np.mean(self.X_train[self.y_train == i], axis = 0)
          self.feature_std[i] = np.std(self.X_train[self.y_train == i], axis = 0)

        print("means")
        for i in range(3):
            print('y=' + str(i), end =' ')
            print(self.feature_means[i])

        print('\n     std')
        for i in range(3):
            print('y=' + str(i), end =' ')
            print(self.feature_std[i])

    ## quetion iii
    def predict(self, X_test):
        """Function computes the normalized posterior probabilities and class \
        predictions for the provided test data.
        
        Parameters:
        -----------
        X_test: ndarray of shape (N,D)
            2D numpy array containing N testing examples having D dimensions each.
               
        Returns:
        --------
        y_pred: ndarray of shape (N,1)
            vector containing class predictions for each of the N training\
             points in X_test.
               
        posteriors: ndarray of shape (N,C)
            numpy array containing normalized class posterior probabilities \
            for each of the C classes for each training example.
                   
        """
        a = (X_test.shape[0], 1, X_test.shape[1])
        my_mean = (1, self.feature_means.shape[0], self.feature_means.shape[1])
        my_std = (1, self.feature_std.shape[0], self.feature_std.shape[1])
        gaussian = self.log_gaussian(X_test.reshape(a), self.feature_means.reshape(my_mean), self.feature_std.reshape(my_std))

        # perform inference
        posteriors = np.exp(np.sum(gaussian, axis = -1) + np.log(self.priors)) 
        posteriors_a = (posteriors / np.sum(posteriors, axis = 1).reshape(-1, 1))
        
        y_pred = np.argmax(posteriors_a, axis = 1)
        
        return y_pred, posteriors_a

#-----------------Don't change anything below------------------------#

# define train and test sizes
N_train = X_train.shape[0]
N_test = X_test.shape[0]

model = MyNaiveBayes(X_train, Y_train)
model.fit()
y_pred, _ = model.predict(X_test)
    
print("Accuracy Score: %.3f" % accuracy_score(Y_test, y_pred))    

means
y=0 [ 53.1275692    0.48972321   0.51616881   0.87914497  17.21512743
  32.6912821   85.40367224 256.84091532   1.13702795]
y=1 [ 49.95196507   0.38864629   0.45851528   0.84279476  16.26637555
  30.93566433  84.60262009 259.12227074   1.35052402]
y=2 [ 44.09574468   0.4787234    0.46808511   0.92553191  16.81914894
  31.57008372  83.57446809 253.68085106   2.63021277]

     std
y=0 [ 9.50736566  0.49989438  0.86357679  0.32595873  8.37890912  4.20488318
  8.62727152 19.63223918  1.23580315]
y=1 [11.9647356   0.48744266  0.70201175  0.36399417  7.99529094  8.06877216
  6.9290977  24.51757915  1.46639227]
y=2 [10.22519744  0.4995471   0.51986348  0.26253112  5.63983319 12.12775823
 10.27337952 24.29876661  2.31756015]
Accuracy Score: 0.414
